In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4914/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_10_49_9,0.996604,0.070873,-0.196916,0.995748,0.000529,0.551570,0.838464,0.000451,0.008130,0.022995,1.000660,0.023174,1601.089968,3875.044787,"Hidden Size=[36, 18], regularizer=0.02, learni..."
1,model_10_49_8,0.996580,0.071068,-0.197458,0.995998,0.000532,0.551454,0.838844,0.000425,0.008144,0.023073,1.000664,0.023252,1601.076383,3875.031202,"Hidden Size=[36, 18], regularizer=0.02, learni..."
2,model_10_49_7,0.996555,0.071266,-0.198006,0.996238,0.000536,0.551337,0.839228,0.000399,0.008159,0.023159,1.000669,0.023339,1601.061445,3875.016264,"Hidden Size=[36, 18], regularizer=0.02, learni..."
3,model_10_49_6,0.996527,0.071462,-0.198554,0.996468,0.000541,0.551220,0.839612,0.000375,0.008174,0.023254,1.000675,0.023435,1601.045132,3874.999951,"Hidden Size=[36, 18], regularizer=0.02, learni..."
4,model_10_49_5,0.996496,0.071663,-0.199106,0.996688,0.000546,0.551101,0.839999,0.000352,0.008189,0.023356,1.000681,0.023538,1601.027523,3874.982342,"Hidden Size=[36, 18], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,model_3_0_1,-0.870052,-0.125356,-0.061044,-1.887349,0.291133,0.668060,0.743284,0.185577,0.260841,0.539568,-2.015459,0.543767,100.467949,240.977138,"Hidden Size=[12], regularizer=0.02, learning_r..."
4855,model_2_0_2,-0.895614,-0.242958,-0.517621,-1.475368,0.295113,0.737874,1.063125,0.226457,0.239836,0.543243,-1.547231,0.547470,68.440797,163.069433,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4856,model_2_0_1,-1.029416,-0.328809,-0.627534,-1.670128,0.315943,0.788838,1.140121,0.244274,0.252355,0.562088,-1.727028,0.566463,68.304385,162.933022,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4857,model_2_0_0,-1.832159,-0.411051,-0.541471,-3.402694,0.440916,0.837661,1.079833,0.402777,0.272655,0.664015,-2.805714,0.669182,67.637803,162.266440,"Hidden Size=[8], regularizer=0.02, learning_ra..."
